In [48]:
#Reto | Deserción de empleados
#Rocio Julian
#1.	Importa las librerías requeridas.

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#2.Lee el archivo CSV llamado empleadosRETO.csv y coloca los datos en un frame de Pandas llamado EmpleadosAttrition.

EmpleadosAttrition = pd.read_csv(R"C:\Users\r30363\OneDrive - Netafim\Documents\Data Scientist\C6. Machine learning\SC15. Ingeniería de las caracteristicas\Reto\empleadosRETO.csv")
                        

In [49]:
#3.Elimina las columnas que, con alta probabilidad (estimada por ti), no tienen relación alguna con la salida. 
#Hay algunas columnas que contienen información que no ayuda a definir el desgaste de un empleado, tal es caso de las siguientes:
#a.	EmployeeCount: número de empleados, todos tienen un 1
#b.	EmployeeNumber: ID del empleado, el cual es único para cada empleado
#c.	Over18: mayores de edad, todos dicen “Y”
#d.	StandardHours: horas de trabajo, todos tienen “80”

EmpleadosAttrition = EmpleadosAttrition.drop(['EmployeeCount','EmployeeNumber','Over18','StandardHours'],axis=1)

In [50]:
#4.	Analizando la información proporcionada, detectaste que no se cuenta con los años que el empleado lleva en la compañía y 
#parece ser un buen dato. 
#Dicha cantidad se puede calcular con la fecha de contratación ‘HiringDate’:
#a.	Crea una columna llamada Year y obtén el año de contratación del empleado a partir de su fecha ‘HiringDate’. 
#No se te olvide que debe ser un entero.
#b.	Crea una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018.
#Para su cálculo, usa la variable Year que acabas de crear.


EmpleadosAttrition['HiringDate'] = pd.to_datetime(EmpleadosAttrition['HiringDate'], errors='coerce')
EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].dt.year
EmpleadosAttrition['YearsAtCompany'] = 2018-EmpleadosAttrition['Year']



In [51]:
#5.	La DistanceFromHome está dada en kilómetros, pero tiene las letras “km” al final y así no puede ser entera:
#a.	Renombra la variable DistanceFromHome a DistanceFromHome_km.
#b.	Crea una nueva variable DistanceFromHome que sea entera, es decir, solo con números

EmpleadosAttrition = EmpleadosAttrition.rename(columns={'DistanceFromHome':'DistanceFromHome_km'})
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.split().str[0].astype(float)


In [52]:
#6.	Borra las columnas Year, HiringDate y DistanceFromHome_km debido a que ya no son útiles.

EmpleadosAttrition = EmpleadosAttrition.drop(['Year','HiringDate','DistanceFromHome_km'],axis=1)


In [53]:
#7.	Aprovechando los ajustes que se están haciendo, la empresa desea saber si todos los departamentos tienen un ingreso promedio similar. 
#Genera una nuevo frame llamado SueldoPromedioDepto que contenga el MonthlyIncome promedio por departamento de los empleados
#y colócalo en una variable llamada SueldoPromedio. 
#Esta tabla solo es informativa, no la vas a utilizar en el set de datos que estás construyendo.

SueldoPromedioDepto = EmpleadosAttrition.groupby("Department")["MonthlyIncome"].mean().reset_index()
SueldoPromedio = SueldoPromedioDepto


In [54]:
#8.	La variable MonthlyIncome tiene un valor numérico muy grande comparada con las otras variables. 
#Escala dicha variable para que tenga un valor entre 0 y 1. 

escalador = preprocessing.MinMaxScaler()
frame_escalado = escalador.fit_transform(SueldoPromedio[["MonthlyIncome"]])
SueldoPromedio['MonthlyIncome_esc'] = frame_escalado



In [55]:
#9.	Todo parece indicar que las variables categóricas que quedan sí son importantes para obtener la variable de salida. 
#Convierte todas las variables categóricas que quedan a numéricas:
#a.	BusinessTravel
#b.	Department
#c.	EducationField
#d.	Gender
#e.	JobRole
#f.	MaritalStatus
#g.	Attrition

EmpleadosAttrition = pd.get_dummies(EmpleadosAttrition, columns=['BusinessTravel', 
                                                                 'Department', 
                                                                 'EducationField', 
                                                                 'Gender', 'JobRole', 
                                                                 'MaritalStatus', 'Attrition'])


In [56]:
#10.	Ahora debes hacer la evaluación de las variables para quedarte con las mejores. 
#Calcula la correlación lineal de cada una de las variables con respecto al Attrition.

if 'Attrition_Yes' in EmpleadosAttrition.columns:
    EmpleadosAttrition['Attrition'] = EmpleadosAttrition['Attrition_Yes']
    EmpleadosAttrition = EmpleadosAttrition.drop(['Attrition_Yes', 'Attrition_No'], axis=1)

correlations = EmpleadosAttrition.corr()['Attrition'].sort_values(ascending=False)



C:\Users\r30363\AppData\Local\Temp\ipykernel_30632\4194818515.py:8: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations = EmpleadosAttrition.corr()['Attrition'].sort_values(ascending=False)


In [57]:
#11.	Selecciona solo aquellas variables que tengan una correlación mayor o igual a 0.1, dejándolas en otro frame 
#llamado EmpleadosAttritionFinal. 
#No olvides mantener la variable de Sali<da Attrition; esto es equivalente a borrar las que no cumplen con el límite.

selected_features = correlations[correlations >= 0.1].index
EmpleadosAttritionFinal = EmpleadosAttrition[selected_features]


In [58]:
#12.	Crea una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal. 
#Recuerda que el resultado del proceso PCA es un numpy array, por lo que, para hacer referencia a una columna
#por ejemplo, la 0, puedes usar la instrucción EmpleadosAttritionPCA[:,0]).

pca = PCA()
EmpleadosAttritionPCA = pca.fit_transform(EmpleadosAttritionFinal)

print("Proporción de varianza explicada por cada componente:", pca.explained_variance_ratio_)
print("Primeros 5 registros del PCA:")
print(EmpleadosAttritionPCA[:5])



Proporción de varianza explicada por cada componente: [0.3722182  0.25923761 0.19637282 0.09303961 0.07913176]
Primeros 5 registros del PCA:
[[-0.3209446  -0.17001888  0.04862275 -0.06323365 -0.03072831]
 [-0.3209446  -0.17001888  0.04862275 -0.06323365 -0.03072831]
 [ 1.13093509  0.11558706  0.60139958  0.44993043 -0.52223114]
 [ 0.58141073 -0.36462858 -0.32327774 -0.0649331   0.06709332]
 [ 0.08355051  0.34375218  0.75542148 -0.33183381 -0.05742135]]


In [59]:
#13.	Agrega el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar 
#el 80% de la varianza, al frame EmpleadosAttritionFinal. 
#Puedes usar la instrucción assign, columna por columna, llamando a cada una C0, C1, etc., hasta las que vayas a agregar.


scaler = StandardScaler()
scaled_data = scaler.fit_transform(EmpleadosAttritionFinal)
pca = PCA()
EmpleadosAttritionPCA = pca.fit_transform(scaled_data)


cumulative_variance = pca.explained_variance_ratio_.cumsum()
num_components = (cumulative_variance >= 0.8).argmax() + 1

print(f"Número mínimo de componentes para explicar el 80% de la varianza: {num_components}")


pca_columns = {f'C{i}': EmpleadosAttritionPCA[:, i] for i in range(num_components)}
pca_df = pd.DataFrame(pca_columns)


EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(**pca_df)


print(EmpleadosAttritionFinal.head())




Número mínimo de componentes para explicar el 80% de la varianza: 4
   Attrition  MaritalStatus_Single  JobRole_Sales Representative  \
0          0                     0                             0   
1          0                     0                             0   
2          1                     1                             1   
3          0                     1                             0   
4          1                     0                             0   

   EducationField_Technical Degree  JobRole_Laboratory Technician        C0  \
0                                0                              0 -0.743180   
1                                0                              0 -0.743180   
2                                0                              0  4.281797   
3                                0                              0  0.352972   
4                                0                              0  0.817579   

         C1        C2        C3  
0 -0.496138  0

In [61]:
#14.	Guarda el set de datos que has formado y que tienes en EmpleadosAttritionFinal en un archivo CSV 
#llamado EmpleadosAttritionFinal.csv. 
#Las últimas columnas que colocaste quedarán después de la variable Attrition, lo cual no importa, 
#pero si gustas lo puedes arreglar antes de escribir el archivo.

pca_columns = [col for col in EmpleadosAttritionFinal.columns if col.startswith('C')]


ordered_columns = [col for col in EmpleadosAttritionFinal.columns if col not in pca_columns + ['Attrition']]
ordered_columns += ['Attrition'] + pca_columns


EmpleadosAttritionFinal = EmpleadosAttritionFinal[ordered_columns]


EmpleadosAttritionFinal.to_csv(R"C:\Users\r30363\OneDrive - Netafim\Documents\Data Scientist\C6. Machine learning\SC15. Ingeniería de las caracteristicas\Reto\EmpleadosAttritionFinal.csv", index=False)

print("El archivo EmpleadosAttritionFinal.csv ha sido guardado exitosamente.")


El archivo EmpleadosAttritionFinal.csv ha sido guardado exitosamente.
